In [ ]:
!conda install -c conda-forge ipywidgets -y

In [1]:
import re
import gzip
import pandas as pd
import pysam
import gffutils

In [ ]:
db = gffutils.create_db(gtf_file, 
                        '/data/gencode.v43lift37.annotation.gtf.db',
                        disable_infer_genes=True, 
                        disable_infer_transcripts=True,
                        keep_order=True
                        )

In [2]:
anno_gff = '/work/CanonicalTrasncripts/gencode.v43lift37.annotation.sort.gff3.gz'
anno_db = '/data/gencode.v43lift37.annotation.gtf.db'
db = gffutils.FeatureDB(anno_db)

In [3]:
with open('./CanonicalTranscripts.exoncount.tsv', mode='x', encoding='utf-8') as outfile:
    columns = ['CHROM', 'Start', 'End', 'Strand', 'GeneSymbol', 'HGNC_ID', 
               'MaxExon', 'ENSG', 'ENST', 'ENSG_Full', 'ENST_Full', 
               'GeneType', 'Tag']
    outfile.write('\t'.join(columns) + '\n')
    with gzip.open(anno_gff) as annos:
        for r in pysam.tabix_iterator(annos, pysam.asGFF3()):
            try:
                r.tag
            except KeyError:
                pass
            else:
                if r.feature == 'transcript':
                    if 'Ensembl_canonical' in r.tag:
                        ensg = re.match(r'ENSG\d+', r.gene_id).group()
                        enst = re.match(r'ENST\d+', r.transcript_id).group()
                        exons = db.children(r.transcript_id, featuretype='exon')
                        exon_counter = 0
                        while 1:
                            try:
                                next(exons)
                            except StopIteration:
                                break
                            else:
                                exon_counter += 1
                        try:
                            r.hgnc_id
                        except KeyError:
                            write_record = [
                                r.contig, str(r.start), str(r.end), r.strand,
                                r.gene_name, '.', str(exon_counter), ensg, enst,
                                r.gene_id, r.transcript_id, r.gene_type, r.tag
                                ]
                            outfile.write('\t'.join(write_record) + '\n')
                        else:
                            hgnc = re.search(r'\d+', r.hgnc_id).group()
                            write_record = [
                                r.contig, str(r.start), str(r.end), r.strand,
                                r.gene_name, hgnc, str(exon_counter), ensg, enst,
                                r.gene_id, r.transcript_id, r.gene_type, r.tag
                                ]
                            outfile.write('\t'.join(write_record) + '\n')
                    else:
                        pass
                    pass
                else:
                    pass